In [44]:
# Modules
import xml.etree.ElementTree as ET
import random
import os
import glob

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
# Local
import parsing_xml as px

In [5]:
exml = ET.parse('tests/latexmled_files/math.0407523.xml')

In [39]:
ns = {'latexml': 'http://dlmf.nist.gov/LaTeXML' }
para_lst_nonrand = exml.findall('.//latexml:para',ns)
para_lst = random.sample(para_lst, 10)

In [40]:
[px.recutext1(par) for par in para_lst]

['       theorem b [corollary   and theorem ]suppose that  and . for all  such that                     ',
 '     proof. (i) is standard, while (ii) follows at once from lemma .',
 'it remains to prove (iv) and (v). when      , the necessity of the conditions () and () has already been proved in lemma , [ and lemma , lemma 4.1]. for , () and () both reduce to , which is a necessary condition for non-emptiness by [., remark 5.7]',
 '     proof. (i) follows from proposition  and corollary . for (ii), note that . finally, for (iii), recall that  and  because  is a principally polarized abelian variety. ∎',
 '         suppose that  and that ( holds. then, ) and  are smooth and irreducible. moreover these varieties are isomorphic outside subvarieties of codimension at least .',
 '(ii) is proved by interchanging the subscripts       in the proof of (i) ∎',
 '             if , .',
 '             if , .',
 '             ,',
 'now the        -semistability criterion implies that  and hence     

In [54]:
out_file = open('../out_rand.txt','a')
ns = {'latexml': 'http://dlmf.nist.gov/LaTeXML' }
for f in glob.glob('../2004_ends_with_3/*/*.xml'):
    try:
        exml = ET.parse(f)
        para_lst_nonrand = exml.findall('.//latexml:para',ns)
        para_lst = random.sample(para_lst_nonrand, 2)
        for p in para_lst:
            out_file.write(px.recutext1(p) + "\n")
    except ET.ParseError:
        pass
    except ValueError:
        print('article %s has few paragraphs'%f)
out_file.close()

article ../2004_ends_with_3/math.0412073/psfrag.xml has few paragraphs
article ../2004_ends_with_3/math.0412073/bibliography.xml has few paragraphs
article ../2004_ends_with_3/math.0412073/preamble.xml has few paragraphs
article ../2004_ends_with_3/math.0412533/gtoutput.xml has few paragraphs


In [48]:
glob.glob('../2004_ends_with_3/*/*.xml')

['../2004_ends_with_3/math.0406533/june25.xml',
 '../2004_ends_with_3/math.0401443/math.0401443.xml',
 '../2004_ends_with_3/math.0404453/math.0404453.xml',
 '../2004_ends_with_3/math.0411373/weakhs2.xml',
 '../2004_ends_with_3/math.0411213/math.0411213.xml',
 '../2004_ends_with_3/math.0402143/math.0402143.xml',
 '../2004_ends_with_3/math.0412103/math.0412103.xml',
 '../2004_ends_with_3/math.0410133/math.0410133.xml',
 '../2004_ends_with_3/math.0405303/math.0405303.xml',
 '../2004_ends_with_3/math.0405073/29juni08goodcomp.xml',
 '../2004_ends_with_3/math.0410203/cluckers_loeser_inv_final.xml',
 '../2004_ends_with_3/math.0403543/ryb-final.xml',
 '../2004_ends_with_3/math.0410283/math.0410283.xml',
 '../2004_ends_with_3/math.0404463/t2new.xml',
 '../2004_ends_with_3/math.0412503/math.0412503.xml',
 '../2004_ends_with_3/math.0407123/math.0407123.xml',
 '../2004_ends_with_3/math.0407143/nbCarreDePoints.xml',
 '../2004_ends_with_3/math.0409243/math.0409243.xml',
 '../2004_ends_with_3/math.04